# Understanding GAN

<img src="images/boxing.jpg">

 * [Generative Adversarial Nets](https://arxiv.org/pdf/1406.2661.pdf)
 * [NIPS 2016 Tutorial:Generative Adversarial Networks](https://arxiv.org/pdf/1701.00160.pdf)
 * [Understanding Generative Adversarial Networks](https://danieltakeshi.github.io/2017/03/05/understanding-generative-adversarial-networks/)
 * [Adit Deshpande - Generative Adversarial Nets](https://adeshpande3.github.io/adeshpande3.github.io/Deep-Learning-Research-Review-Week-1-Generative-Adversarial-Nets)
 
해당 문서를 보기전에 공부해야할 문서들
 * [Khan Academy - Game theory and Nash equilibrium](https://www.khanacademy.org/economics-finance-domain/microeconomics/nash-equilibrium-tutorial)

# How do GANs work

GAN의 아이디어는 기본적으로 2개의 모델이 서로 경쟁하듯 학습을 하는 원리입니다. <br>
그중 하나는 **Generator**로서 training data와 동일한 분포형태(distribution)을 갖는 샘플을 생성합니다. <br>
다른 하나는 **Discriminator**로서 샘플 데이터가 real 인지 또는 fake인지 구분합니다. <br>
즉 **2개의 classes** (real or fake)를 갖은 traditional supervised learning을 사용합니다. 

쉽게 이야기하면 Generator는 가짜돈을 찍어내는 도둑놈이고, Discriminator는 가짜돈인지 진짜돈인지 구별해내는 경찰의 역활입니다.<br>
게임에서 이기기 위해서 도둑놈(Generator)은 진짜 돈과 구별할수 없는 가짜 돈을 만들어야 합니다.

도둑놈과 경찰은 2개의 differentiable functions으로 표현이 됩니다. 

| Name | Function | Cost | Input | Parameter |
|:-----|:--------:|:-----:|:---------:|
| Distriminator | $ D $ | $ \theta^{(D)} $ 만 control하면서 minimize $ J^{(D)} \left( \theta^{(D)}, \theta^{(G)} \right) $ | $ x $ | $ \theta^{(D)} $ | 
| Generator     | $ G $ | $ \theta^{(G)} $ 만 control하면서 minimize $ J^{(G)} \left( \theta^{(D)}, \theta^{(G)} \right) $ | $ z $ | $ \theta^{(G)} $ |

위의 표안의 Cost부분에서 나와있듯이, 각각의 player의 cost값은 다른 player의 parameter에 의존하고 있지만, <br>
상대방의 parameters를 건들일수 없기 때문에, 시나리오 자체가 optimization problem보다는 game으로 설명이 더 잘 됩니다.

Optimization문제의 해결책은 cost값의 (local) minimum을 찾는 것입니다.<br>
Game에서의 해결책은 Nash equilibrium입니다. (Local diffenrential Nash equilibria)

<img src="images/gan.png">

* $ D(x) $ 의 목표는 input으로 받은 데이터가 real인지에 대한 (fake가 아니라) 확률값을 output으로 내놓으며 1값으로 만드는게 목표
* $ x $의 절반은 data samples(real)이고, 절반은 generator가 생성산 samples(fake)입니다.
* $ D(G(z)) $ -> latent variable $ z $로부서 generator 가 x sample과 유사한 distribution을 output으로 내놓고, 해당 output은 discriminator가 input $ G(z) $로 받습니다.
* Discriminator는 $ D(G(z)) $를 0으로 만들려고 하며, Generator는 1로 만들려고 합니다.

# Training Process

Simultaneous SGD를 사용하며, 각각의 step마다 2개의 minibatches가 sampled 됩니다. <br>
minibatch of $ x $는 dataset으로 부터 sampled되며, minibatch of $ z $는 latent variables로부터 만들어지게 됩니다. 

이후 2개의 gradient steps이 동시에 일어나게 됩니다.<br>
하나는 $ \theta^{(D)} $를 업데이트하면서 $ J^{(D)} $를 줄입니다.<br>
다른 하나는 $ \theta^{(G)} $를 업데이트하면서 $ J^{(G)} $의 loss를 줄입니다. 

# Cost Functions

### Discriminator's cost, $ J^{(D)} $

Discriminator의 loss function은 binary cross entropy loss function을 사용합니다. 

$$ H((x_1,y_1),D) = - y_1 \log D(x_1) - (1-y_1) \log (1-D(x_1)) $$

위의 공식은 하나의 data point에 대해서 처리를 할때 사용하며, 전체 데이터세는 다음의 공식을 사용합니다.

$$ H((x_i,y_i)_{i=1}^N,D) = - \sum_{i=1}^N y_i \log D(x_i) - \sum_{i=1}^N (1-y_i) \log (1-D(x_i)) $$ 

GAN에서 $ x_i $는 반은 $ x_i \sim P_{\rm data} $ (즉 true data distribution) 그리고 반은 $ x_i = G(z) $ 이때 $ z \sim p_{\rm generator} $ (generators' distribution)에서 옵니다.<br>
반반이라는 뜻은 $ z \sim {\rm Unif}[0,1] $ 입니다.

dataset으로부터 오는 minibatch의 경우 모두 1이고, generator로부터 오는 minibatch는 모두 0입니다.<br>
정확하게 2개의 minibatch를 사용하기 때문에 GAN논문에서는 다음과 같이 cross entropy를 변경하였습니다.

$$ H((x_i,y_i)_{i=1}^\infty,D) = - \frac{1}{2} \mathbb{E}_{x \sim p_{\rm
data}}\Big[ \log D(x)\Big] - \frac{1}{2} \mathbb{E}_{z} \Big[\log
(1-D(G(z)))\Big] $$


$ \sum $ 은 expectation으로 변경하였으며, $ y_i $ label 은  $ \frac{1}{2} $으로 변경하였습니다. <br>
또한 $ \log(1-D(x_i)) $ 는 $ \log(1-D(G(z))) $ 으로 변경하였습니다.


## Loss Function

Discriminator $ D $의 loss function은 기본적으로 binary cross entropy를 사용합니다. <br>
이때 $ D(x_i) \in [0, 1] $ 즉.. D함수의 output은 0 ~ 1사이의 값입니다.

$$ H((x_i,y_i)_{i=1}^N,D) = - \sum_{i=1}^N y_i \log D(x_i) - \sum_{i=1}^N (1-y_i) \log (1-D(x_i)) $$



## Discriminator's Cost

$$ J^{(D)}\left( \theta^{(D)}, \theta^{(G)} \right) = -\frac{1}{2} \mathbb{E}_{x \sim Pdata} $$